In [1]:
import os
os.chdir("C:/Users/venkat sai/Desktop/InternshipProject/ECG arrhythmia images")

# Data preprocessing

In [2]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.4,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [4]:
x_train = train_datagen.flow_from_directory('Training_dataset',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                     class_mode = 'categorical')
x_test = test_datagen.flow_from_directory('Test_dataset',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')



Found 2600 images belonging to 5 classes.
Found 651 images belonging to 5 classes.


# importing the model building libraries

In [5]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Model initialization

In [6]:
model=Sequential()

# Adding convolution layers

In [7]:
model.add(Convolution2D(48,(3,3),input_shape=(64,64,3),activation="relu"))

In [8]:
#adding pooling layers
model.add(MaxPooling2D(pool_size=(2,2)))

In [9]:
#adding Flatten layers
model.add(Flatten())

# Adding dense layers

In [10]:
#hidden layer
model.add(Dense(init="uniform",activation="relu",output_dim=150))

C:\Users\venkat sai\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=150, kernel_initializer="uniform")`
  


In [11]:
#output layer
model.add(Dense(init="uniform",activation="softmax",output_dim=5))

C:\Users\venkat sai\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=5, kernel_initializer="uniform")`
  


# Configuring Model

In [12]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [13]:
#class indices
x_train.class_indices

{'Fusion_beat': 0,
 'Normal_beat': 1,
 'Supraventricular_ectopic_beat': 2,
 'Unknown_beat': 3,
 'Ventricular_ectopic_beat': 4}

# Fit model or training model 

In [14]:
model.fit_generator(x_train, steps_per_epoch=500, epochs=12,validation_data=x_test)


Epoch 1/12
500/500 [==============================] - 105s 210ms/step - loss: 1.0712 - accuracy: 0.5778 - val_loss: 2.0070 - val_accuracy: 0.2673
Epoch 2/12
500/500 [==============================] - 88s 175ms/step - loss: 0.6232 - accuracy: 0.7765 - val_loss: 2.1158 - val_accuracy: 0.3195
Epoch 3/12
500/500 [==============================] - 88s 175ms/step - loss: 0.5272 - accuracy: 0.8131 - val_loss: 1.6473 - val_accuracy: 0.3763
Epoch 4/12
500/500 [==============================] - 87s 175ms/step - loss: 0.4552 - accuracy: 0.8463 - val_loss: 3.1079 - val_accuracy: 0.3886
Epoch 5/12
500/500 [==============================] - 86s 173ms/step - loss: 0.3966 - accuracy: 0.8633 - val_loss: 4.0549 - val_accuracy: 0.3917
Epoch 6/12
500/500 [==============================] - 85s 169ms/step - loss: 0.3326 - accuracy: 0.8834 - val_loss: 2.2939 - val_accuracy: 0.3625
Epoch 7/12
500/500 [==============================] - 84s 168ms/step - loss: 0.2813 - accuracy: 0.9063 - val_loss: 4.6183 - val_

In [15]:
model.save("ECG.h5")

In [16]:
from keras.models import load_model
import numpy as np

In [17]:

model = load_model('ECG.h5')
model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

from skimage.transform import resize

def detect(frame):
    try:
        beat_type=['Fusion beat','Normal beat','Supraventricular ectopic beat','unknown beat','Ventricular ectopic beat']
        img = resize(frame,(64,64))
        img = np.expand_dims(img,axis=0)
        if(np.max(img)>1):
            img = img/255.0
        prediction = model.predict(img)
        print(prediction)
        prediction = model.predict_classes(img)
        print(beat_type[prediction[0]])
    except AttributeError:
        print("shape not found")


In [18]:
import cv2
path=r'C:\Users\venkat sai\Desktop\InternshipProject\ECG arrhythmia images\Test_dataset\Normal_beat\fig_72522.png'

In [19]:
frame=cv2.imread(path)
data=detect(frame)

[[8.3955085e-01 7.9069622e-02 6.6135600e-02 1.5222409e-02 2.1601621e-05]]
Fusion beat


In [20]:
path=r'C:\Users\venkat sai\Desktop\InternshipProject\ECG arrhythmia images\Test_dataset\Fusion_beat\Ffig_686.png'
data=detect(cv2.imread(path))

[[8.8534391e-01 1.4119419e-08 2.9556299e-04 2.9732203e-03 1.1138722e-01]]
Fusion beat


In [21]:
data=detect(cv2.imread('Afig_2298.png'))

shape not found


In [22]:
path=r'C:\Users\venkat sai\Desktop\InternshipProject\ECG arrhythmia images\Test_dataset\Supraventricular_ectopic_beat\Afig_2289.png'
data=detect(cv2.imread(path))

[[0.40035048 0.00801364 0.5839494  0.0040755  0.00361103]]
Supraventricular ectopic beat


In [23]:
path=r'C:\Users\venkat sai\Desktop\InternshipProject\ECG arrhythmia images\Test_dataset\Unknown_beat\pfig_31203998.png'
detect(cv2.imread(path))

[[9.9852788e-01 4.2025473e-08 3.1460324e-04 1.0757456e-03 8.1813392e-05]]
Fusion beat


In [24]:
path=r'C:\Users\venkat sai\Desktop\InternshipProject\ECG arrhythmia images\Test_dataset\Normal_beat\fig_72600.png'
detect(cv2.imread(path))

[[8.0753785e-01 1.0629938e-03 6.7274667e-02 1.2408304e-01 4.1463521e-05]]
Fusion beat


In [25]:
path=r'C:\Users\venkat sai\Downloads\ventricular_beat.png'
detect(cv2.imread(path))

[[9.979955e-01 7.930958e-08 9.539769e-04 6.992992e-05 9.805947e-04]]
Fusion beat


In [26]:
path=r'C:\Users\venkat sai\Downloads\supraventricular_beat.jpg'
detect(cv2.imread(path))

[[7.6450169e-07 2.2305735e-09 9.9739420e-01 2.6030047e-03 2.0091520e-06]]
Supraventricular ectopic beat


In [27]:
detect(cv2.imread("test1.png"))

[[9.979955e-01 7.930958e-08 9.539769e-04 6.992992e-05 9.805947e-04]]
Fusion beat
